In [27]:
# !pip install pymupdf

   ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
    --------------------------------------- 0.3/18.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.4 MB 2.6 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/18.4 MB 2.5 MB/s eta 0:00:07
   --- ------------------------------------ 1.8/18.4 MB 2.4 MB/s eta 0:00:07
   ----- ---------------------------------- 2.4/18.4 MB 2.4 MB/s eta 0:00:07
   ----- ---------------------------------- 2.6/18.4 MB 2.4 MB/s eta 0:00:07
   ------ --------------------------------- 3.1/18.4 MB 2.4 MB/s eta 0:00:07
   ------- -------------------------------- 3.7/18.4 MB 2.4 MB/s eta 0:00:07
   --------- ------------------------------ 4.2/18.4 MB 2.4 MB/s eta 0:00:07
   ---------- ----------------------------- 4.7/18.4 MB 2.4 MB/s eta 0:00:06
   ----------- ---------------------------- 5.2/18.4 MB 2.4 MB/s eta 0:00:06
   ------------ --------------------------- 5.8/18.4 MB 2.4 MB/s eta 0:00:06
   ----------

In [31]:
import pymupdf

doc = pymupdf.open("HasilCPNS2024KemendikbudLampiran1.pdf")
print(doc.page_count)

16071


In [33]:
page = doc[0]
text = page.get_text("text")
print(text)

PANITIA SELEKSI NASIONAL PENGADAAN CASN 2024
Jabatan Formasi
JF0000334
ANALIS HUKUM AHLI PERTAMA
10
:
-
Jumlah
Kode
Instansi
3010
Kementerian Pendidikan, Kebudayaan, Riset dan Teknologi
12843
:
-
10
KEMENTERIAN PENDIDIKAN, KEBUDAYAAN, RISET, DAN TEKNOLOGI | Sekretariat Jenderal | Biro Hukum
30102461
Lokasi Formasi
:
-
Jenis Formasi
1
UMUM
7
:
-
Pendidikan
S-1 HUKUM
7
REKAP HASIL INTEGRASI SKD DAN SKB
PENGADAAN CPNS 2024
Jumlah
Peserta
SKB
Jumlah
Formasi
Jumlah Metode
SKB
Hadir
Tidak Lulus
Tidak Hadir
TMS
Lulus Akhir
SKB
1
2
3
4
5
6
7
10
20
7
1
19
1
13
0
7
Peserta
0
TMS-1
8
0
9
APS
Hasil
Laporan digenerate secara otomatis melalui Aplikasi Pengolahan Nilai SSCASN Badan Kepegawaian Negara, Panitia Seleksi Nasional © 2024
1  /  16071



In [35]:
blocks = page.get_text("blocks")

In [ ]:
def get_lines(page):
    h_lines = []
    v_lines = []

    for d in page.get_drawings():
        for item in d["items"]:
            if item[0] == "l":  # line
                x0, y0, x1, y1 = item[1]
                if abs(y0 - y1) < 1:
                    h_lines.append((x0, y0, x1, y1))
                elif abs(x0 - x1) < 1:
                    v_lines.append((x0, y0, x1, y1))

    return h_lines, v_lines

In [ ]:
def get_table_bbox(h_lines, v_lines):
    xs = [x for l in v_lines for x in (l[0], l[2])]
    ys = [y for l in h_lines for y in (l[1], l[3])]

    if not xs or not ys:
        return None

    return min(xs), min(ys), max(xs), max(ys)


In [ ]:
def extract_table_text(page, bbox):
    words = page.get_text("words")
    table_words = [
        w for w in words
        if bbox[0] <= w[0] <= bbox[2]
        and bbox[1] <= w[1] <= bbox[3]
    ]
    return table_words


In [ ]:
def group_rows(words, tol=5):
    words = sorted(words, key=lambda w: (w[1], w[0]))
    rows = []

    for w in words:
        if not rows or abs(rows[-1][0][1] - w[1]) > tol:
            rows.append([w])
        else:
            rows[-1].append(w)

    return rows


In [ ]:
import pandas as pd

def rows_to_df(rows):
    data = []
    for row in rows:
        row = sorted(row, key=lambda w: w[0])
        data.append([w[4] for w in row])
    return pd.DataFrame(data)


In [ ]:
import os

output_dir = "pymupdf_tables"
os.makedirs(output_dir, exist_ok=True)

for i in range(doc.page_count):
    page = doc[i]

    h_lines, v_lines = get_lines(page)
    bbox = get_table_bbox(h_lines, v_lines)

    if not bbox:
        continue

    words = extract_table_text(page, bbox)
    rows = group_rows(words)

    if rows:
        df = rows_to_df(rows)
        df.to_csv(f"{output_dir}/page_{i+1}.csv", index=False)
